In [1]:
%pip install --upgrade --quiet langchain-nvidia-ai-endpoints

Note: you may need to restart the kernel to use updated packages.


In [74]:
# # https://docs.api.nvidia.com/nim/reference/visual-models-apis
# ChatNVIDIA.get_available_models()

In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

llm = ChatNVIDIA(model="adept/fuyu-8b",
                api_key = '')


In [84]:
import base64
from mimetypes import guess_type

In [85]:
class ImageAnalyst:
    def __init__(self):
        self.client = llm

    # Function to encode a local image into data URL
    def local_image_to_data_url(self, image_path):
        # Guess the MIME type of the image based on the file extension
        mime_type, _ = guess_type(image_path)
        if mime_type is None:
            mime_type = 'application/octet-stream'  # Default MIME type if none is found

        # Read and encode the image file
        with open(image_path, "rb") as image_file:
            base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

        # Construct the data URL
        return f"data:{mime_type};base64,{base64_encoded_data}"

    def analyseImage(self, data_url):
        system_prompt = """
          You are an expert media analyst. Your expertise includes analyzing and understanding every detail that is present in the media.
          Make sure to include every small details and dynamic objects such as content on the parking signs, traffic signals, historic sites, road signs, and animals.

          Your Task:
          Carefully examine the following media in detail and provide a detailed, factual, and accurate explanation of what the media depicts.
          If this is a well-known location, please provide me with every relevant detail.
          Highlight all the key elements and their significance, and present your analysis in clear, well-structured markdown format.
        """

        response = self.client.invoke(
            [
                        {"role": "system",
                         "content": system_prompt
                         },
                        {"role": "user",
                         "content": [
                             {
                                 "type": "text",
                                 "text": "Describe this image: "
                             },
                             {
                                 "type": "image_url",
                                 "image_url":
                                     {
                                         "url": data_url
                                     }
                             }
                         ]}
                    ],
            max_tokens = 1024
        )

        return response.content

    def pipe(self, image_path):
        data_url = self.local_image_to_data_url(image_path)
        response = self.analyseImage(data_url=data_url)
        return response


In [86]:
imgAt_obj = ImageAnalyst()

# Pass the image to the pipeline function
file_path = 'input.jpg'
image_analysis_result = imgAt_obj.pipe(file_path)

print(image_analysis_result)

Exception: [422] Unprocessable Entity
body -> messages
  List should have at most 1 item after validation, not 2 (type=too_long; field_type=List; max_length=1; actual_length=2)
RequestID: f9a25739-a2da-4fff-8e5b-7a1d4fae4bd8